In [ ]:
!pip install yfinance scipy --quiet

import yfinance as yf
from scipy.stats import norm
import numpy as np
import pandas as pd
from datetime import datetime

# Black-Scholes 공식 (콜)
def bs_call(S, K, T, r, sigma):
    if T <= 0: return max(S - K, 0)
    d1 = (np.log(S/K) + (r + sigma**2 / 2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

# 기본 세팅
ticker = "AAPL"
r = 0.052
sigma = 0.25
gap_threshold = 0.5  # 괴리 기준 (예시)
fee_estimate = 0.15  # 수수료/슬리피지 등 감안

# 데이터 수집
stock = yf.Ticker(ticker)
S = stock.history(period="1d", interval="1m")['Close'].iloc[-1]
expiration_str = stock.options[0]
expiration = datetime.strptime(expiration_str, "%Y-%m-%d")
T = (expiration - datetime.today()).days / 365
calls = stock.option_chain(expiration_str).calls

# 분석 결과 저장
results = []

for idx, row in calls.iterrows():
    K = row['strike']
    market = row['lastPrice']
    if np.isnan(market) or market < 0.1:
        continue

    theo = bs_call(S, K, T, r, sigma)
    diff = market - theo

    if abs(diff) > gap_threshold:
        strategy = "CALL 매도 (고평가)" if diff > 0 else "CALL 매수 (저평가)"
        expected_gain = abs(diff) - fee_estimate
    else:
        strategy = "-"
        expected_gain = 0

    results.append({
        "Strike": K,
        "Market Price": market,
        "Theo Price (σ=0.25)": round(theo, 2),
        "Diff": round(diff, 2),
        "전략": strategy,
        "예상수익(추정)": round(expected_gain, 2)
    })

# 결과 출력
df = pd.DataFrame(results).sort_values("Strike")
df_trade = df[df["전략"] != "-"]

from IPython.display import display
print("전체 분석 결과 (전략 추천 포함):")
display(df_trade)
